# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Student 1 - BUI QUANG PHUONG Linh [qbuiquan@ulb.ac.be](mailto:qbuiquan@ulb.ac.be) - Student ID : 000427796
### Student 2 - SINGH Sundeep [susingh@ulb.ac.be](mailto:student2@ulb.ac.be) - Student ID : 000428022

## --- Kaggle competition ---


# Introduction


As part of the INFO-F422 course, this project is tackling a classification problem which implies different methods of feature selection and models implementation to make predictions. The main purpose is a comparison between those different methods to find which one fits and predicts with the most accuracy. The dataset "train.csv" which is used to train the model contains 13083 elements described by 46 features including the "target" feature which is our prediction objective. 

Firstly, different methods of feature selection are implemented to extract the most interesting features in the dataset, which depends of different criteria according to the methods used. In particular, filter methods such as correlation with the output, mRMR as well as wrapper methods are used.   

Then, the obtained results in the feature selection will be used to create the predictive models and predict the target following the best extracted features. More precisely, the models implemented are Support Vector Machines (SVM), Random Forests (RF) and Decision Trees. 

Finally, a combination of different models strategies is carried out to achieve a better overall performance on prediction's accuracy.   

### Running time
The <b>whole</b> code (if you try to run the code with "Cell" -> "Run All") takes about ~20 min to be executed and show all the results.

# Dataset preprocessing and libraries

In [1]:
options(warn=-1)

Install the packages if needed: 

In [2]:
#install.packages("e1071")
#install.packages("rpart")
#install.packages("randomForest")

Import libraries that will be needed to run the code. 
- <b>rpart</b>: library that contains Decision Trees model's methods
- <b>e1071</b>: library that contains SVM model's methods
- <b>randomForest</b>: library that contains Random Forests model's methods

In [3]:
library(e1071)
library(rpart)
library(randomForest)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.


In case of NA values which are representating missing datas, it may cause troubles in the next steps of prediction such as computing correlation between two variables. To handle this, the following function is replacing those NA values by the mean value of the column (feature). 

In [4]:
# Pre-processing function that replaces all NA values in the dataset by the mean column value 
replace_na_with_mean_value<-function(vec) {
  mean_vec<-mean(vec,na.rm=T)
  vec[is.na(vec)]<-mean_vec
  vec
}

First, we initialize the training "train.csv" and testing "test.csv" dataset and preprocess the training set using the previous function. Then, we split in two the training set : 
- $X$ : contains the whole dataset without the target feature. X is used to train the model with the aim of obtaining the results in Y. 
- $Y$ : contains only the target feature column. Y is used to compare the real target values with the predicted one. 

In [5]:
# Initializing the required data sets

full_training <- read.csv("train.csv")
test_set <- read.csv("test.csv")
full_training<-data.frame(apply(full_training,2,replace_na_with_mean_value))

set.seed(3)

X<-full_training[,setdiff(colnames(full_training),"target")] # Removes the "target" column that we want to predict 
Y<-full_training[,"target"] # Contains only the "target" column

N<-nrow(X)    # Number of elements (examples)
n<-ncol(X)    # Number of features 

# 1 - Feature selection


A feature selection is the process to extract the most relevant subset of features to make the best accurate prediction. Indeed, without this process, the overall model performance tends to decrease with too many inputs especially when learning irrelevant or redundant informations. This problem is well-known and is called <i>overfitting</i>.   

Thereby, the feature selection process facilitates data visualization and data understanding by reducing the measurement, storage requirements and training time. Thus, it also proposes a solution to the curse of dimensionality which consists of a phenomena that arise when analyzing and organizing data in high-dimensional spaces which will not occurs in a lower dimensionality. However, this process takes some computation time and then increases the algorithmic complexity. 

Subsequently, it exists different methods of feature selection : 
- <b>Filter methods:</b> process used to measure the relevance of the features. The basic idea behind filter methods is to measure the model performance while not using the obtained subset through the process. In filter criteria, all the features are scored and ranked based on certain statistical criteria. The features with the highest ranking values are selected and the low scoring features are removed. Generally, one of the main disadvantage of filter methods is that each feature is considered independently thus ignoring feature dependencies. Some examples of filter methods are principal component analysis (PCA), mRMR, variance thresholds, etc.  

</br> 

- <b>Wrapper methods:</b> compared to filter methods, wrapper methods measure the usefulness of the features. Herein, what we are looking for is a the optimal feature subset to get the best performance using the learning algorithm itself as part of the evaluation function. The evaluation of a specific subset of features is obtained by training and testing a specific classification model. Nevertheless, a huge drawback of these methods is that the complexity is growing exponentially to the feature's number because of the search algorithm to search the space of all feature subsets. 

## Filter methods 

### Correlation with the output

In this method, the feature relevance is depending of the correlation between the features values and the target values. The correlation is the relationship between these variables. A positive correlation means that both variables move in the same direction while a negative correlation means that when one variable's value increases, the other variables' values decrease. Nevertheless, in this case, we don't consider these two types of correlation. Whether it is a positive or negative correlation, it is considered as a correlation factor. Thus, we take the absolute values of the correlation value to compare the importance (positive or negative) of the feature variable. Therefore, <b>greater is this correlation, more the feature is relevant for the predictive model</b>.  

#### Correlation function 
By default, the $cor$ function from R is using the <i>Pearson Correlation Coefficient</i>. This method for correlation is computed following this formula: 
$$ \rho = \frac{\sum_{i=1}^n (x_{i} - \bar{x})(y_{i} - \bar{y})}{(n-1) \sigma_{x} \sigma_{y}} $$
where $n$ is the number of elements, $x_{i}$ the $i^{th}$ features value, $y_{i}$ the $i^{th}$ target value and $\sigma$ the standard deviation. 

#### 10-fold cross-validation
Cross-validation (fully called $k$-fold cross-validation) is one of the most known method which is estimating the skill of machine learning models. In standard $k$-fold cross-validation, we partition the data into $k$ subsets, called folds. Then, we iteratively train the algorithm on $k−1$ folds while using the remaining fold as the test set. In this case, the features selection procedure use a 10-folds cross-validation procedure in order to make the ranking. More precisely, the procedure is a resampling method used in order to see how our model performs when some data is missing. Here, we create two different data set one for training and another one for testing. Consequently, we resample the training set each folds and verify the correlation of the features with the target. 

In [6]:
size.CV<-floor(N/10)
CV.err<-matrix(0,nrow=n,ncol=10)

# 10-fold cross-validation procedure 
for (i in 1:10) {
  
  # Testing set
  i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
  X.ts<-X[i.ts,]         
  Y.ts<-Y[i.ts]  
    
  # Training set
  i.tr<-setdiff(1:N,i.ts)                
  X.tr<-X[i.tr,] 
  Y.tr<-Y[i.tr]                          
  
  correlation<-abs(cor(X.tr,Y.tr)) # Compute the correlation between the features and the target 
  ranking<-sort(correlation,decreasing=TRUE,index.return=T)$ix # Sort the features from the most correlated one to the least  

}

#### Remark concerning the indexes : 
The index $i$ of the ranking corresponds to the feature whose first column name is $N(i-1)$ because of the "id" column that is also considered as a feature and corresponds to the index 1. Thus, for example, the index 21 corresponds to the feature N20.

In [7]:
ranking

[1] 21 28 25  7 30 27 12 31 19 26 20 32  9 36 33 46  3 39 44 24 29 43 23 13 35
[26] 22 15 37 38  1 34 17  8 11 14  2 41 40  5  4  6 45 16 42 18 10

In [8]:
correlation

id,0.0301205960
N1,0.0173303097
N2,0.0580562983
N3,0.0079206310
N4,0.0089693323
N5,0.0075886052
N6,0.0978932162
N7,0.0231060763
N8,0.0665630992
N9,0.0001310887
N10,0.0220398258


### Top 30 features with correlations with the output 

In order to use the best features found for prediction, here is the 30 best features. These 30 features will be given as data to create the predictive model. Note that this value of 30 is not arbitrary. Indeed, we have tried to predict in the next section with different numbers of top features: 10, 15, 20, 25, 30 and 35. Overall, 30 seems to be the best number of features to get the best performance. That is the reason why we have chosen to take the 30 top features to create the models in the next section.     

In [9]:
ord <- ranking[0:30]
x30 <- X.tr[,ord]
names(x30)

[1] "N20" "N27" "N24" "N6"  "N29" "N26" "N11" "N30" "N18" "N25" "N19" "N31"
[13] "N8"  "N35" "N32" "N45" "N2"  "N38" "N43" "N23" "N28" "N42" "N22" "N12"
[25] "N34" "N21" "N14" "N36" "N37" "id"

### mRMR

This method creates a ranking by considering the correlations with the output but adds some precision to it. More precisely, mRMR will also avoid redudant varibles by adding a phase of post processing to the method of "correlation with the output". Therefore, the method will take the ranking made and substract a redundance score to it which is computed as the mean of the correlation between one feature and all the other features. Then, the feature left with the max value after the substraction is taken as a top feature and we do the same with this feature. In other words, we take this feature and compute the redundancy score accompagnied by the substraction until we get our new top ranking. More formally, the computation of the redundancy $W_{c}$ is done following this formula: 
$$ W_{c} = \frac{1}{n} \sum_{i,j} | c(i,j) | $$ 
where $n$ is the number of features (columns) and $c(i,j)$ the correlation between the variable $i$ and $j$. 

To summarize, the signification of mRMR does it well: <b>m</b>inimum <b>R</b>edundancy <b>M</b>aximum <b>R</b>elevance. 

In [10]:
size.CV<-floor(N/10)

CV.err<-matrix(0,nrow=n,ncol=10)

# 10-fold cross-validation process
for (i in 1:10) {
    
    # Testing Set 
    i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
    X.ts<-X[i.ts,]  
    Y.ts<-Y[i.ts]  
    
    # Training Set
    i.tr<-setdiff(1:N,i.ts)
    X.tr<-X[i.tr,]
    Y.tr<-Y[i.tr]
    
    mRMR.correlation<-abs(cor(X.tr,Y.tr)) # Compute the correlation between the features and the target
    
    selected<-c()
    candidates<-1:n # The candidates list are all the features but after every round the best feature is removed
    
    #mRMR ranks the variables by taking account not only the correlation with the output, but also by avoiding redudant variables
    for (j in 1:n) {
        redudancy.score<-numeric(length(candidates)) 
        if (length(selected)>0) {
            cor.selected.candidates<-cor(X.tr[,selected,drop=F],X.tr[,candidates,drop=F])
            redudancy.score<-apply(cor.selected.candidates,2,mean) #update redundancy score
        }
        
        mRMR.score<-mRMR.correlation[candidates]-redudancy.score #update mrMR score
        
        selected_current<-candidates[which.max(mRMR.score)]
        selected<-c(selected,selected_current)
        candidates<-setdiff(candidates,selected_current) # remove best feature from candidates 
    }
    
    mRMR.ranking<-selected
}  

In [11]:
mRMR.ranking

[1] 21 28 30 26 27 29 25 13 17  7 32 36 12 41 19  3 46 24 33 38 45  2 44 34 40
[26] 31  9  1 15 35 43  5 22 20 16 42 23 39  4  8 10 11 14 18  6 37

In [12]:
mRMR.correlation

id,0.0301205960
N1,0.0173303097
N2,0.0580562983
N3,0.0079206310
N4,0.0089693323
N5,0.0075886052
N6,0.0978932162
N7,0.0231060763
N8,0.0665630992
N9,0.0001310887
N10,0.0220398258


### Top 30 features with mRMR

In [13]:
ord_mRMR <- mRMR.ranking[0:30]
x30_mRMR <- X.tr[,ord_mRMR]
names(x30_mRMR)

[1] "N20" "N27" "N29" "N25" "N26" "N28" "N24" "N12" "N16" "N6"  "N31" "N35"
[13] "N11" "N40" "N18" "N2"  "N45" "N23" "N32" "N37" "N44" "N1"  "N43" "N33"
[25] "N39" "N30" "N8"  "id"  "N14" "N34"

### Results interpretation and methods comparison

As we can see, the ranking obtained in both filtering methods differs. This is due to additional phase done by the mRMR method which is characterized by the fact that if a feature is too much related with another one then is considered as a redundant variable thus the ranking will be changed. For instance, if we consider the second and third top features in the method "correlation with the output" and the mRMR method we can see that the third top feature isn't the same. In "correlation with the output" it is N24 but for mrMR it is N29. This is due to the fact that the feature N27 has a correlation of 0.1464617760 and the feature N24 has a correlation 0.1424325042 which is very similar and then considered has a redundant variable. Finally, we can see that in mRMR method, feature N24 will be outclassed by feature N29 that is less redundant with a correlation of 0.0845777330. To sum up, the general behaviour of both rankings are then quite similar excepting some features that are considered as redundant when their correlation score are very close which are therefore regressed in the top feature ranking. 

### Summary of filter methods

The two filter methods that we have tackled here are complementary depending of the information that we want to extract. mRMR can somehow be seen as an enhancement of "correlation with output" method since it adds a post-processing method which remove features redundancies. Nevertheless, sometimes it could be interesting to not remove these redundancies which may remove an important desired feature. In this case, the selection of features with filter methods is independent of any machine learning algorithms. This is one of the reasons why no learning algorithm are used in the implemented filter methods algorithms. 

Instead, features are then selected on the basis of their scores in various statistical tests for their correlation with the outcome variable. The pseudo-code below represents the process of mRMR method which contains the "correlation with output" method (method 1) followed by the post-processing method which consists of removing redundancy. 

#### Pseudo-Code of filter methods
   
    X <- put all features except target
    Y <- put only target
    Init(size.CV, CV.err)

    FOR(i TO number of folds in cross validation process):
    DO:
        X.tr, Y.tr <- createTrainingSet(X,Y)
        X.ts, Y.ts <- createTestingSet(X,Y)
        computeCorrelation(X.tr, Y.tr)
        
        # For "Correlation with output" method: Make the ranking regarding the correlations at this step. 
        
        # For mRMR: Removal of redundancy process below. 
        
        selectedFeature <- createEmptyList
        candidates <- length(allFeatures)

        FOR(j to numberOfFeatures):
        DO:
            computeRedudancyScore(candidates)
            IF(there is a topFeature selected):
            DO:
                correlation <- computeCorrelation(topFeature against all other features left)
                computeNewRedudancyScore(correlation(candidates), redudanceScore)
                
            mRMRscore <- correlationOfAllCandidates - redudancyScore
            topFeatureSelected <- feature with highest correlation
            RemoveFromCandidates(topFeatureSelected)

## Wrapper methods

Wrapper methods constitutes the second main category of feature selection methods. Comparing the filter methods, the main difference is that the learning algorithm is an entire part of the process. This difference is especially notable regarding to the computation time. Compared to filter methods, wrapper methods take way more time due to its high complexity. Indeed, wrapper methods is a searching problem which consists of finding the best features subset using a model to learn. To do that, different types of wrapper methods exist: 
- <b>Forward Selection:</b> this is an iterative method that starts from an empty subset to finally obtain the best ranking features subset. At each iteration, the feature that improves the best (which means that implies the lowest error rate) our model is added and stops until adding a feature does not improve the performance of the model or simply when there is no features left in case of only doing a features ranking.
- <b>Backward Elimination:</b> somehow the contrary of Forward Selection. Here we start with the full set of features and removes the least significant feature at each iteration. We stop when removing any feature isn't improving the model. 
- <b>Recursive Feature elimination:</b> as its name said, this method repeatedly creates models and keeps aside the best or the worst performing feature at each iteration. The next model is constructed with these left features. This is then a recursive process which stops when all features are eliminated. Thus, the ranking is done in order of their elimination.

In this document, the algorithm implemented is Forward Selection using Linear Regression as learning algorithm. All the main steps are highlighted in comments of the code. 
A summary of the wrapper methods process is summarized in the following figure (source: Saurav Kaushik (2016)): 
![Wrapper methods process](figures/wrapper.PNG)

### Forward Selection

<b>Note:</b> The Forward Selection algorithm takes a little bit of time to execute. Please be patient. 

In [14]:
size.CV<-floor(N/10)

selected<-NULL

# For every feature (column)
for (round in 1:n) { 
  candidates<-setdiff(1:n,selected)
  
  CV.err<-matrix(0,nrow=length(candidates),ncol=10)
  
  # For every candidates   
  for (j in 1:length(candidates)) {
    features_to_include<-c(selected,candidates[j])
    
    # 10-fold cross-validation process  
    for (i in 1:10) {
        
      # Testing Set   
      i.ts<-(((i-1)*size.CV+1):(i*size.CV))  
      X.ts<-X[i.ts,features_to_include,drop=F]  
      Y.ts<-Y[i.ts]  
      
      # Training Set   
      i.tr<-setdiff(1:N,i.ts)
      X.tr<-X[i.tr,features_to_include,drop=F]
      Y.tr<-Y[i.tr]
      
      DS<-cbind(X.tr,target=Y.tr)
      
      # Learning algorithm
      model<- lm(target~.,DS) # Linear model
      Y.hat.ts<- predict(model,X.ts)
      
      # Error computation (Evaluation)
      CV.err[j,i]<-mean((Y.hat.ts-Y.ts)^2)
    }
  }

  CV.err.mean<-apply(CV.err,1,mean)
  CV.err.sd<-apply(CV.err,1,sd)
    
  # Get the feature with the minimum error which is the feature that improves the model the best 
  selected_current<-which.min(CV.err.mean)    
  selected<-c(selected,candidates[selected_current])
    
  wrap.ranking<-selected  
  print(paste("Rank ",round," ; Selected feature: ",candidates[selected_current]," ; CV error=",round(CV.err.mean[selected_current],digits=4), " ; std dev=",round(CV.err.sd[selected_current],digits=4)))
}

[1] "Rank  1  ; Selected feature:  21  ; CV error= 0.2441  ; std dev= 0.0064"
[1] "Rank  2  ; Selected feature:  25  ; CV error= 0.2395  ; std dev= 0.0069"
[1] "Rank  3  ; Selected feature:  28  ; CV error= 0.236  ; std dev= 0.0074"
[1] "Rank  4  ; Selected feature:  27  ; CV error= 0.2328  ; std dev= 0.0084"
[1] "Rank  5  ; Selected feature:  18  ; CV error= 0.2313  ; std dev= 0.0072"
[1] "Rank  6  ; Selected feature:  20  ; CV error= 0.2284  ; std dev= 0.0072"
[1] "Rank  7  ; Selected feature:  29  ; CV error= 0.2268  ; std dev= 0.0083"
[1] "Rank  8  ; Selected feature:  7  ; CV error= 0.2261  ; std dev= 0.0092"
[1] "Rank  9  ; Selected feature:  13  ; CV error= 0.2256  ; std dev= 0.0086"
[1] "Rank  10  ; Selected feature:  19  ; CV error= 0.2249  ; std dev= 0.0085"
[1] "Rank  11  ; Selected feature:  44  ; CV error= 0.2247  ; std dev= 0.0084"
[1] "Rank  12  ; Selected feature:  11  ; CV error= 0.2245  ; std dev= 0.0086"
[1] "Rank  13  ; Selected feature:  31  ; CV error= 0.2242  ; s

In [15]:
wrap.ranking 

[1] 21 25 28 27 18 20 29  7 13 19 44 11 31  3  8 23 36 43 45 10 12 37  9 42 34
[26] 46 38 41 40  2 22 30 26 32 33 15 16 24 35  5 39 14  1 17  4  6

### Top 30 features with Forward Selection

In [16]:
x30_wrap <- X.tr[0:30]
names(x30_wrap)

[1] "N20" "N24" "N27" "N26" "N17" "N19" "N28" "N6"  "N12" "N18" "N43" "N10"
[13] "N30" "N2"  "N7"  "N22" "N35" "N42" "N44" "N9"  "N11" "N36" "N8"  "N41"
[25] "N33" "N45" "N37" "N40" "N39" "N1"

### Results interpretation and comparison with filter methods

The ranking obtained differs from the one obtained by the filter methods. This can be explained by the fact that a learning algorithm is used to establish the ranking. More precisely, at the first step we consider all the features and select the one wich performs the best in our prediction i.e. which will have the minimum CV.error, here we can see that it is the feature 21. Once this feature selected, we take this feature and we combine it with all the other features to see how well the combination of the two performs. As we can see the second best feature will be 25 because the combination of the feature 21 with 25 is the one giving the minimum CV.error. Then, we take the previous feature selected, here 25, and combine it to all others and we do the algorithm until we don't have any features left. Finally we can see that the feature 4 for example, tends to perform horribly because the only combination possibe in the end is with the feature 6 which also perfom not well.

## Feature selection conclusion: filter methods vs wrapper methods

In conclusion, here are a table representing the main differences and the advantages (+) and drawbacks (-) of using such method or another. 

| Filter methods  | Wrapper methods    |
|------------|-------------|
|  Measure the relevance of features by their correlation    |     measure the usefulness of a subset of feature by actually training a model on it   |
|  (+) Much faster as they do not involve training the models    |  (-) computationally very expensive      |
|  (-) Might fail to find the best subset of features   |   (+) might provide the best subset of features     |
|  (-) Assume no dependency between the evaluated features   |   (+) evaluation take into account the information dependency between the evaluated features     |
|  (-) No relationship with the classifier   |   (+) better for classification problem because the classifier itself as evaluation criteria     |
|  (+) Thanks to its fastness, suitable for applications that mostly deal with the storage and retrieval of high dimensional datasets.   |   (-) Can need a large number of applications and tests to train to outperform filters    |

# 2 - Model selection


In this section, we are asked to create 3 different model selection procedure to compare their performance. The three following models implemented are Random Forest, SVM and Decision Trees. To create those models, we are using the results obtained in the previous section in the feature selection (FS), i.e. the top 30 features found previously. Thus, a comparison of the model's performance using the different methods of feature selection is also done. 

#### Pre-processing of the data

In [17]:
# Convert training set to the good format 
trainSet <- as.character(Y.tr)
trainSet <- as.factor(trainSet)

## Decision Trees

Decision tree is a model used for classification but also regression used mostly in supervised learning methods. Basically, the tree is contitued of nodes and leaves wherein the solutions to a problem are the leaves and the nodes represent a certain decision to make in other words each nodes represent a certain feature. The advantages from using a decision are the following:
- a better visulation/comprehension of the data
- non linear relationship from data doesn't affect the performance
- it is fast
- it can handle numerical and categorial variable

and the disadvantages from such a model are:
- overfitting
- the variance in the data can create unstable tree 
- a biased tree can be created if a certain class/feature dominate



### Using correlation with output for FS 

In [18]:
# Create the model with Decision Trees method using correlation with output (filter) for feature selection
rpart.model <- rpart(trainSet~., data = x30)

# Prediction 
rpart.predTrain <- predict(rpart.model, X.ts, type = "class")

# Compute accuracy of the model
acc <- mean(rpart.predTrain == Y.ts) 
print(paste0("Accuracy of the model on training dataset: ", acc))

[1] "Accuracy of the model on training dataset: 0.619266055045872"


### Using mRMR for FS 

In [19]:
# Create the model with Decision Trees method using mRMR (filter) for feature selection
rpart.model <- rpart(trainSet~., data = x30_mRMR)

# Prediction 
rpart.predTrain <- predict(rpart.model, X.ts, type = "class")

# Compute accuracy of the model
acc <- mean(rpart.predTrain == Y.ts) 
print(paste0("Accuracy of the model on training dataset: ", acc))

[1] "Accuracy of the model on training dataset: 0.631498470948012"


### Using Forward Selection for FS 

In [20]:
# Create the model with Decision Trees method using Forward Selection (wrapper) for feature selection
rpart.model <- rpart(trainSet~., data = x30_wrap)

# Prediction 
rpart.predTrain <- predict(rpart.model, X.ts, type = "class")

# Compute accuracy of the model
acc <- mean(rpart.predTrain == Y.ts) 
print(paste0("Accuracy of the model on training dataset: ", acc))

[1] "Accuracy of the model on training dataset: 0.622324159021407"


## Random Forest

Random forest is a model used for regression and classification problem. More precisely, it is an aggregation of decision trees (see figure below from Jocelyn D'Souza (2018) which illustrates random forest with 2 decision trees) which will produce a more generalized model such that we avoid the overfitting problem that could result from a simple decision tree. To put simply, in a random forest, multiple decision trees are constructed and each decision tree will contain only a subset of the features the purpose here is to make higly uncorraleted trees because the average error of a random ensemble of errors is zero. Finally it will do as much random split as possible in the trees such that we get a better prediction.

![Random forests](figures/randomForest.PNG)

### Using correlation with output for FS 

In [21]:
# Create the model with randomForest method using correlation with output (filter) for feature selection
RF.model <- randomForest(trainSet~.,data = x30,ntree=1000)

# Prediction 
RF.predTrain <- predict(RF.model, X.ts, type = "class")

# Compute accuracy of the model
acc <- mean(RF.predTrain == Y.ts) 
print(paste0("Accuracy of the model on training dataset: ", acc))

[1] "Accuracy of the model on training dataset: 0.672018348623853"


### Using mRMR for FS 

In [22]:
# Create the model with randomForest method using mRMR (filter) for feature selection
RF.model <- randomForest(trainSet~.,data = x30_mRMR,ntree=1000)

# Prediction 
RF.predTrain <- predict(RF.model, X.ts, type = "class")

# Compute accuracy of the model
acc <- mean(RF.predTrain == Y.ts) 
print(paste0("Accuracy of the model on training dataset: ", acc))

[1] "Accuracy of the model on training dataset: 0.675076452599388"


### Using Forward Selection for FS 

In [23]:
# Create the model with randomForest method using Forward Selection (wrapper) for feature selection
RF.model <- randomForest(trainSet~.,data = x30_wrap,ntree=1000)

# Prediction 
RF.predTrain <- predict(RF.model, X.ts, type = "class")

# Compute accuracy of the model
acc <- mean(RF.predTrain == Y.ts) 
print(paste0("Accuracy of the model on training dataset: ", acc))

[1] "Accuracy of the model on training dataset: 0.67737003058104"


## SVM

Support vector machines (SVM) are supervised learning models that are efficient to resolve classication problems or regression problems. SVM is based on the idea of finding an hyperplane that best separates the features into different
domains. SVMs are a generalization of linear classifiers. Moreover, SVM is the only linear model which can classify data which is not linearly separable as shown in the figure below (source: 2017, Haydar Ali Ismail, Medium.com). The main reason to use an SVM is then when the problem might not be linearly separable. In that case, we will have to use an SVM with a non linear kernel (e.g. RBF).

![SVM](figures/svm.png)

<b>Note:</b> The SVM model takes a little bit of time to execute. Please be patient. 

### Using correlation with output for FS 

In [24]:
# Create the model with SVM method using correlation with output (filter) for feature selection
svm.model <- svm(trainSet~., data = x30, kernel = "linear", type = "C-classification",scale = FALSE)

# Prediction 
svm.predTrain <- predict(svm.model, X.ts, type = "class")

# Compute accuracy of the model
acc <- mean(svm.predTrain == Y.ts) 
print(paste0("Accuracy of the model on training dataset: ", acc))

[1] "Accuracy of the model on training dataset: 0.562691131498471"


### Using mRMR for FS 

In [25]:
# Create the model with SVM method using mRMR (filter) for feature selection
svm.model <- svm(trainSet~., data = x30_mRMR, kernel = "linear", type = "C-classification",scale = FALSE)

# Prediction 
svm.predTrain <- predict(svm.model, X.ts, type = "class")

# Compute accuracy of the model
acc <- mean(svm.predTrain == Y.ts) 
print(paste0("Accuracy of the model on training dataset: ", acc))

[1] "Accuracy of the model on training dataset: 0.652140672782875"


### Using Forward Selection for FS 

In [26]:
# Create the model with SVM method using Forward Selection (wrapper) for feature selection
svm.model <- svm(trainSet~., data = x30_wrap, kernel = "linear", type = "C-classification",scale = FALSE)

# Prediction 
svm.predTrain <- predict(svm.model, X.ts, type = "class")

# Compute accuracy of the model
acc <- mean(svm.predTrain == Y.ts) 
print(paste0("Accuracy of the model on training dataset: ", acc))

[1] "Accuracy of the model on training dataset: 0.675076452599388"


## Results interpretation and comparison

The results obtained for each method is summed up in the table below:

| FS method ↓ | Decision Trees | Random Forest | SVM |
|------|------|------|------|
| Correlation with output  | 61.93% | 67.20%| 56.27% |
| mRMR  | 63.15% | 67.50% | 65.21% |
| Forward Selection  | 62.23%| 67.73% | 67.5% |

#### Comparing Feature Selection methods
The general behaviour that we can remark is that all these accuracies are around 60%. Nevertheless, depending of the methods used, they are fluctuating a lot. Generally, as theoritically said in the previous section, using wrapper methods for classification enhances the accuracy. This assertion is somehow verified. Indeed, the only case where the best accuracy is not obtained when using Forward Selection for feature selection is for the decision trees model. 
Concerning the mRMR feature selection method, it should upgrade the results compared to "correlation with output" method since it doesn't use redundant features. We see that it works pretty well for SVM since the accuracy increased from 56% to 65%. Moreover, it also increases for the 2 other methods, sign that redundant features are overall flawing the classification. 
In summary, for such classification problem, the best feature selection methods to use are ordered as following: Forward Selection (Wrapper) > mRMR (Filter) > Correlation with output (Filter). 

#### Comparing learning methods
As we can see, random forest performs better than decision trees because it is an ensemble of multiple decision trees. Since it uses the bagging method (weak learner are coupled to produce a strong learner) and use always a different subset of the features in each decision tree, it avoids the overfitting problem and produce less variance of the estimation. Thereafter, we can see that random forest also performs better than SVM. These results can be explained by the fact that our dataset contains too much data. More precisely, random forests tend to perform better on large datasets whereas SVM seems to do better in small dataset (less than 10000 rows). Indeed, the only method of the third performing better using less features as data is SVM: around 64% using the top 10 (instead of 30) features using "Correlation with output" method. 

Furthermore, comparing the computational times of these different methods, decision trees leads obviously based on this criteria. Nevertheless, as seen before, this is the fastest method but the less accurate one. Then, comparing random forests and SVM, random forest seems to perform faster on this huge dataset since SVM is often worthwhile training on a smaller dataset first, and tuning your model's hyper-parameter's. Of course, the complexity of random forests is based on the number of trees chosen and could increase a lot and could exceed SVM if using more trees. 

In conclusion, the methods can overall be ordered as following in term of <b>accuracy</b>: Random Forests > SVM > Decision Trees. Concerning <b>computation times</b>, it is ordered as: Decision Trees > Random Forests > SVM. So that, to get a good balance between both, random forests seems to be the best method to adopt over the three presented. 

# 3 - Combination of models

In this section, we are trying to combine models to enhance our prediction accuracy. These models can be the same type such as using multiple decision trees but can also combine different types of models. The idea behind that technique is similar to combining the knowledge of different experts, called <i>weak learners</i>, to become a unique one called <i>strong learner</i>. This combination of models is also called <i>ensemble learning</i>. Basically, the simple ensemble techniques are based on averaging (or weighted average) the results but can also be based on a vote system where the majority vote (i.e. a vote = a model's result) is taken in account. Thus, ensemble learning is a technique that helps to reduce noise, variance, and bias. More accuratly, the two main techniques of ensembling are called <b>bagging</b> and <b>boosting</b>. However before understanding these techniques, the notion of <i>bootstrapping</i> has to be understood.

Bootstrap refers to random sampling with replacement. In other words, we choose $n$ observations (rows) where each row is selected with replacement from the original dataset so that each row is equally likely (same probabilities) to be selected in each iteration which can help to better understand the mean and standard deviation from the dataset. Now that bootstrap is understood, we can define bagging and boosting techniques. 

- <b>Bagging:</b> Bagging is the application of the Bootstrap procedure to a high-variance machine learning algorithm, typically decision trees. In other words, once we have the bootstrap samples, we build a model for each sample. Finally, the  results of these multiple models are combined using average or majority voting. In particular, random forests use this method combining several decision trees.  


- <b>Boosting:</b> Boosting is an iterative technique which adjusts the weight of an observation based on the last classification. Therefore, unlike bagging where each model is acting independently, here each model that runs dictates what features the next model will focus on. Thus, the first algorithm is trained on the entire dataset and the subsequent algorithms are built by fitting the residuals of the first algorithm. 

### AdaBoost implementation

Here below, we are implementing the <b>AdaBoost</b> algorithm which is a boosting algorithm. At first, we will produce a weak classifier (the misclassification error rate is slightly better than random). More precisely, the model will be created based on a dataset sample given a certain probability based on weights. In the beginning, the weights are all the same but updated at each iteration. Once the prediciton done, we will compute the error of prediction given the real dataset and this error will be used to update the weight $a$: 
$$a_j = log(\frac{1 - MME_{emp}^j}{MME_{emp}^j})$$

which will give us the importance of the model at step $j$. The importance will then be used to update the weights $w$:
$$w_i = w_i*exp(alpha_j * MME_j)$$

herein, the weights is the one associated to each observations thus it will put more emphasis on the misclassified observations such that the next iteration will create a sample focusing more on these observation. In other words, by updating these weights we will try to do a better classification than before thus creating in the end a better model by doing a weighted majority vote:
$$model_{boosted} = sign(\sum^m_{j=1} a_j h_j(x,a_n)$$

which will produced the final predicition. 

In [32]:
full_training<- data.frame(apply(full_training,2,replace_na_with_mean_value))
nsample <- nrow(full_training)
full_training[full_training==0] <- -1
halfsample <- floor(nrow(full_training)/2)
Nsample<-nrow(full_training)
full_trainings <- full_training[sample(nrow(full_training)),]
train <- full_trainings[1:halfsample, ]
test <- full_trainings[(halfsample+1):Nsample, ]
Ntrain <- dim(train)[1]
Ntest  <- dim(test)[1] 
w<-rep(1/Ntrain,Ntrain)

# first compute a regular tree misclassification rate
set.seed(555)

# Create the model with Decision Trees method
rpart.model <- rpart(target~., data = train)

# Prediction 
pred <- sign(predict(rpart.model, test))

misc.tree <- sum(as.integer(test$target != sign(pred)))/length(pred)

# boosting initialisation
T<-15

wmisc<-rep(NA,T);
alpha<-rep(NA,T)
pred.test<-rep(0,Ntest)

for (t in 1:T)
{
  set.seed(700+t)
  
  # generate new sample set accoring to their weights
  I<-sample(seq(1,Ntrain),prob=w,replace=TRUE)
  
  # Build a new tree accordingly
  rpart.model <- rpart(target~., data = train[I,])

  # predict
  pred.train  <- sign(predict(rpart.model,train))
  wmisc[t] <- sum(w*as.integer(train$target != pred.train))/sum(w)
    
  # update weigths
  alpha[t]<-log((1-wmisc[t])/wmisc[t])
  w<- w*exp(alpha[t]*as.integer(train$target!= pred.train))
  
  # update predictions
  pred.test<-sign(pred.test+alpha[t]*predict(rpart.model,test))
}

# compute overall boosting misclassification rate
misc.boosting <- mean(test$target != sign(pred.test))


cat("Misclassification single tree=",misc.tree,"\n")
cat("Misclassification boosting=",misc.boosting,"\n")

Misclassification single tree= 0.4105778 
Misclassification boosting= 0.4020177 


<b>Note</b>: when running the above cell for the first time, the results shown are 0.397 missclassification rate for one tree and 0.415 for multiple trees for an unknown reason. If we rerun the cell, we always obtain the same results presented in the following section (0.4105778 for single tree missclassification and 0.4020177 for multiple trees missclassification). 

## Results interpretation

Due to the AdaBoost algorithm explained above we can see that there is less misclassification with 15 trees in opposite of 1 tree. Indeed, we started from 0.4105778 rate of missclassification when using one tree to 0.4020177 when using multiple trees. This optimisation can be due to the fact that the weight are correctly updated on the observation that were missclassified thus focusing more on these observations such that it results in a better classification. 

# 4 - Submission to Kaggle's competition

We have submitted our prediction to the Kaggle's competition and have obtained a score of <b>0.66804</b> (66.804% accuracy, before final results) by using Random Forests learning algorithm and Forward Selection for Feature Selection. Our group name in the leaderboard is "Bui & Singh". 

# Conclusions

In conclusion, we have first implemented 3 methods of feature selection: correlation with output, mRMR and Forward Selection where the 2 first are filter methods and the last one is wrapper methods. Subsequently, we have used the top features returned by these 3 methods to apply it to the 3 predictive models that we have implemented: Decision Trees, Random Forests and SVM. The different results obtained have shown that Forward Selection was globally the best feature selection method for a classification problem. Moreover, comparing the performance of these 3 models, random forests has a constant high accuracy (around 67%) whether the feature selection method used and performs better overall. Finally, we have used the AdaBoost algorithm as combination of models technique to try to enhance the overall performance. In this part, we have tried to predict with a single decision tree and compared it to the prediction with 15 decisions tree. The observation done is that the missclassification rate is a bit less significant than the prediction while using one tree. 